# Esercizio 7 - Frutta con la M

In questo esercizio il sistema estrarrà una lettera a caso e chiederà all'utente di inserire un frutto con la lettera scelta, il 
frutto deve essere inserito in lingua inglese siccome usiamo wordnet per il controllo.

Abbiamo gestito l'esercizio come un gioco in cui l'utente deve battere la macchina che lo interroga.

### Approccio

L'approccio utilizzato è il seguente:
- si parte da una lista di frutti passata in input al sistema;
- per ognuno di questi frutti si individuano i suoi iperonimi;
- per ogni iperonimo si vanno a salvare tutti i suoi iponimi;
- a questo punto viene creata una lista di possibili risposte, che comprende i frutti che iniziano con la lettera scelta.

Il sistema può essere esteso per riconoscere altre categorie oltre ai frutti

### Imports

In [1]:
from nltk.corpus import wordnet as wn
import string
import random

Questa lista contiene le parole con cui il sistema si andrà ad "informare" per capire quali nodi rappresentano dei frutti in WordNet

In [2]:
CATEGORIES = ["FRUIT", "cranberry", "apple", "peanut", "orange"]

### Funzioni

In [3]:
def get_synset(word):
    if(len(wn.synsets(word)) > 0):
        return wn.synsets(word)[0]
    return None

def get_hypers(word):
    syn = get_synset(word)
    hypers = []
    if(syn is not None):
        hypers = syn.hypernyms()
    return hypers

def get_all_hypers(word):
    syn = get_synset(word)
    hypo_list = []
    while(syn.hypernyms() != []):
        hypo_list.append(syn.hypernyms()[0])
        syn = syn.hypernyms()[0]
        
def get_words_cat_hypers(category):
    hypers = []
    for word in category[1:]:
        hypers.append(get_hypers(word))
        
    hypers = [x for xs in hypers for x in xs]
    return hypers

def get_all_hypos(syns):
    hypos = []
    for syn in syns:
        hypos.append(get_hypos(syn))
    hypos = [x for xs in hypos for x in xs]
    return hypos

def get_hypos(syn):
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([get_synset(w) for s in syn.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    return hypo_list

def clean_response(syns):
    for syn in syns:
        fruit = wn.synset('fruit.n.01')
        lch = fruit.lowest_common_hypernyms(syn)
        if(lch[0] != fruit):
            syns.remove(syn)
    return syns

def pick_words(syns, letter):
    answer = []
    for syn in syns:
        lemma = syn.lemma_names()[0]
        if lemma[0] == letter:
            answer.append(lemma)

    return answer


### Esecuzione

In [4]:
def game():
    print("Welcome to the Game!")

    letter = random.choice(string.ascii_letters).lower()
    
    print("Tell me a fruit starting with letter ...", letter.capitalize())

    user_response = input().lower()
    
    hyper_list = get_words_cat_hypers(CATEGORIES)
    
    right_syn = get_all_hypos(hyper_list)
    
    right_syn = clean_response(right_syn) # controllo aggiuntivo 
    
    possible_answers = pick_words(right_syn, letter)

    if(user_response in possible_answers):
        print("Good Job! The answer", user_response, "is correct!")
    elif len(possible_answers) != 0:
        print("\nWrong Answer!", "\n\nYou could say: ", random.choice(possible_answers))
    else:
        print(f"\nYou are very very wise!, There are no fruits starting with letter {letter}")
game()

Welcome to the Game!
Tell me a fruit starting with letter ... Y

Wrong Answer! 

You could say:  yucca


### Limiti del sistema

- Il sistema, così come è realizzato, basa la sua ricerca sugli esempi passati in input, che quindi sono i principali responsabili delle sue prestazioni.
- WordNet è una risorsa gerarchica che non fa uso del senso comune, concetto che sarebbe utile per trattare un gioco di questa natura.
- Inoltre, WordNet è molto specifico (ci sono frutti sconosciuti), e i nodi non sono organizzati per essere reperiti facilmente per tale scopo.